In [1]:
import os

datadir = "/Users/michielk/oxdata/P01/EM/Myrf_01/SET-B/B-NT-S10-2f_ROI_00/zws"
# pred_file = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs1_eed2_main.h5')  # dataset: 'main'
pred_file = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_main_vol00.h5')  # dataset: 'main'
aff_file = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_main_vol00_grad.h5')  # dataset: 'main'
out_folder = os.path.join(datadir, 'zws_vol00_')
outname = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_zws.h5')  # dataset: 'main'
max_len = 300


In [5]:
# in conda root env
import h5py
import numpy as np
from wmem import utils

h5path_in = os.path.join(pred_file, 'main')
h5file_in, ds_in, elsize, axlab = utils.h5_load(h5path_in)
grad = np.array(np.absolute(np.gradient(ds_in[0,:,:,:], 1)))

h5file_in.close()

h5path_out = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_main_vol00_absgrad.h5', 'main')
h5file_out, ds_out = utils.h5_write(None, grad.shape, grad.dtype,
                                    h5path_out,
                                    element_size_um=elsize,
                                    axislabels=axlab)
ds_out[:] = grad
h5file_out.close()


In [ ]:
from zwatershed import (partition_subvols,
                        eval_with_par_map,
                        eval_with_spark,
                        stitch_and_save,
                        merge_by_thresh)


In [ ]:
partition_data = partition_subvols(aff_file, out_folder, max_len)


In [ ]:
eval_with_spark(partition_data[0])
# NUM_WORKERS=4
# eval_with_par_map(partition_data[0], NUM_WORKERS)


In [ ]:
stitch_and_save(partition_data, outname)


In [ ]:
%matplotlib nbagg

import numpy as np
import sys
import time
import os
import h5py
import os.path as op
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from multiprocessing import Pool
from itertools import product
from zwatershed import (partition_subvols,
                        eval_with_par_map,
                        stitch_and_save,
                        merge_by_thresh)
# from par_funcs import *
# sys.path.append('..')
cmap = matplotlib.colors.ListedColormap(np.vstack( ((0, 0, 0), np.random.rand(1e6, 3))) )
V = 20

datadir = "/Users/michielk/oxdata/P01/EM/Myrf_01/SET-B/B-NT-S10-2f_ROI_00/zws"
# zwsbase = os.path.join(datadir, "zws")
outname = os.path.join(datadir, 'B-NT-S10-2f_ROI_00ds7_probs_zws.h5')  # dataset: 'main'


In [ ]:
orig_file = h5py.File(aff_file,'r')
start = np.array([0, 0, 0])
stop = np.array([191, 301, 244])
fig, axs = plt.subplots(1, 3)
for i in range(0, 3):
    orig = orig_file['main'][i, start[0]:stop[0], start[1]:stop[1], start[2]:stop[2]]
    ax = axs[i]
    cax = ax.imshow(orig[V,:,:], cmap=plt.get_cmap('Greys'))
    cbar = fig.colorbar(cax, ax=ax, orientation='horizontal')
    ax.set_title('orig{}'.format(i))
orig_file.close()
plt.show()


In [ ]:
num, thresh = 0, 1000.0
fig, axs = plt.subplots(1, 2)

basic_file = h5py.File(os.path.join(datadir, 'zws_vol01_0_0_0_vol', 'basic.h5'),'r')
seg_init = np.array(basic_file['seg'])
rg_init = np.array(basic_file['rg'])
keeps = rg_init[:,0]<rg_init[:,1]
rg_init = rg_init[keeps,:]
seg_sizes_init = np.array(basic_file['counts'])
basic_file.close()
ax = axs[0]
ax.imshow(seg_init[V,:,:], cmap=cmap)
ax.set_title('seg_init')

f = h5py.File(outname, 'a')
s,e = f['starts'][num],f['ends'][num]
seg = f['seg'][s[0]:e[0]-3,s[1]:e[1]-3,s[2]:e[2]-3]
seg_sizes = np.array(f['seg_sizes'])
rg = np.array(f['rg_'+str(num)])
f.close()
ax = axs[1]
ax.imshow(seg[V,:,:], cmap=cmap)
ax.set_title('seg_after_stitching')

plt.show()

print "num_segs",len(np.unique(seg_init)),len(np.unique(seg))
print "rg lens",len(rg_init),len(rg)


In [ ]:
num, thresh = 0, 0.5
seg_init_merged = merge_by_thresh(seg_init, seg_sizes_init, rg_init, thresh)
seg_merged = merge_by_thresh(seg, seg_sizes, rg, thresh)


In [ ]:
fig, axs = plt.subplots(1, 2)

ax = axs[0]
ax.imshow(seg_init_merged[V,:,:], cmap=cmap)
ax.set_title('merged init')

ax = axs[1]
ax.imshow(seg_merged[V,:,:], cmap=cmap)
ax.set_title('merged')

plt.show()

print "num_segs",len(np.unique(seg_init)),len(np.unique(seg))
print "rg lens",len(rg_init),len(rg)
